## **Chapter 7 - [Ensemble Learning & Random Forests](https://github.com/ageron/handson-ml2/blob/master/07_ensemble_learning_and_random_forests.ipynb)**

Este algoritmo se asimila a lo que llamamos "la sabiduría de las masas", es decir, que la predicción agregada de varios predictores (SVM, Regrsiones, Arboles de Decisión, etc) pueden entregar un mejor resultado que la predicción de un experto. 

Este método hace lo mismo, une a varios predictores. Por tanto, será usual entrenar una serie de muy buenos modelos, luego agregarlos a través de un método de ensemble y así obtener un resultado todavía mejor. Hacer click [aquí](https://www.netflixprize.com/) para más curiosidades sobre esta metodología de Machine Learning.

In [0]:
#Importamos Librerías Típicas
import numpy as np
import matplotlib.pyplot as plt

***Datos***

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=500, noise=0.30, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

### **7.1 VotingClassifiers**

####***a. Estimadores***

Aquí utilizamos todo el poder múltiples clasificadores. Con ellos podremos unir lo bueno y separar lo malo de cada uno. Esto ocurre porque cada Algoritmo analiza patrones de una manera específica, al ser así, cada uno está sujeto a perder información que otros sí consideran. 

Por tanto, realizar la estimación de manera unida provoca que dichas debilidades se "mitiguen" siempre y cuando los errores sean independientes. Esto último es imposible siempre que entrenemos al algoritmo con los mismo datos. Sin embargo, podemos salvarnos de esto utilizando algoritmos MUY distintos.

El funcionamiento en términos más duros es a través de ver las probabilidades que cada clasificador le asigna a cada observación. En base a dichos valores el VotingClassifier analiza cuales son las clasificaciones más seguras.

cambiandoles desde "hard" a "soft" muchas veces obtendremos mejores resultados, ya que lo hecho en el último parráfo tendrá mayor relevancia.


In [0]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf = LogisticRegression(solver="lbfgs")
rnd_clf = RandomForestClassifier(n_estimators=100)
svm_clf = SVC(probability=True, gamma="scale")

voting_clf = VotingClassifier(estimators=[("lr", log_clf), ("rf", rnd_clf), ("svc", svm_clf)],
                             voting="soft")

####***b. Entrenamos el modelo***

In [4]:
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)), ('rf', RandomF...',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False))],
         flatten_transform=None, n_jobs=None, voting='soft', weights=None)

####***c. Vemos que tan bueno es***

Aquí nos interesa saber como le va a cada clasificador por separado, así podemos ver el verdadero poder de ***VotingClassifier***

Los resultados hablan solos, esta vez se ocupó "soft" en la metodología de votos. Esto porque obtiene mejores resultados.

In [5]:
from sklearn.metrics import accuracy_score
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
  clf.fit(X_train, y_train)
  y_pred = clf.predict(X_test)
  print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.864
RandomForestClassifier 0.896
SVC 0.896
VotingClassifier 0.912


### **7.2 Bagging & Pasting**

***Bagging***: Consiste en tomar muestras aleatorias a partir de la muestra de entrenamiento. Y con ellas realizar las estimaciones. Así se espera obtener un resultado más cercano al valor verdadero
***Pasting***: Lo mismo, pero cambia una cosa. Las observaciones que se sacan de la muestra para cada submuestra no se vuelve a repetir.

El resultado final de esta metodología es uno agregado de cada predicción en la submuestra. Este tendrá un sesgo similar, pero con menor varianza que si entrenasemos un unico predictor sobre la muestra.

Estos métodos son muy utiles para muestras grandes.

In [6]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators=500,
                           max_samples=100, bootstrap=True, n_jobs=1)
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.912

Lo bueno de esta metodología es que generaliza mucho mejor que realizar un arbol de decisión solo. Esto porque las predicciones son hechas sobre datos más diversos, ya que cada sub muestra añade nueva información que de lo contrario sería desapercibida.

#### **a. Evaluación Out-of-Bag **

Con Bagging algunas de las observaciones es probable que no sea usadas para crear submuestras. Esto no lo queremos, ya que no queremos perder información.

Pero no todo es malo, ya que estas observaciones nunca antes vistas pueden servir de validación para el modelo.

In [7]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators=500,
                           max_samples=100, bootstrap=True, n_jobs=1, oob_score=True)
bag_clf.fit(X_train, y_train)
bag_clf.oob_score_

0.9253333333333333

### **7.3 Random Forests**

Es un Arbol de Decisión hecho con metodología Bagging, aportando mayor diversidad a la estimación, ya que no se busca el estimador con mejor fit, sino que el mejor estimador entre muestras aleatorias. Lo último suma mayor sesgo, pero menor varianza. Entregando como resultado mejores estimaciones usualmente.



In [10]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=1)
rnd_clf.fit(X_train, y_train)

y_pred_rf = rnd_clf.predict(X_test)
y_pred_rf

array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0,
       0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0,
       1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0])

Adelante, tenemos un algoritmo que basicamente hace lo mismo que el anterior. Un Arbol de Decisión con Bagging como metodología. 

In [0]:
bag_clf = BaggingClassifier(DecisionTreeClassifier(splitter="random", max_leaf_nodes=16),
                           n_estimators=500, max_samples=1.0, bootstrap=True, n_jobs=1)


In [32]:
bag_clf.fit(X_train, y_train)
y_pred_bag = bag_clf.predict(X_test)
y_pred_bag

array([1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0,
       0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0,
       1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0])

###**7.4 Feature Importance**

Un método de RandomForest que le aporta gran interpretabilidad al algoritmo. A través de dar a conocer cuanta impureza resta cada atributo al modelo general.

In [11]:
from sklearn.datasets import load_iris
iris = load_iris()
rnd_clf = RandomForestClassifier(n_estimators=500, n_jobs=1)
rnd_clf.fit(iris["data"], iris["target"])
for name, score in zip(iris["feature_names"], rnd_clf.feature_importances_):
  print(name, score)

sepal length (cm) 0.09852921942033192
sepal width (cm) 0.02283996462832791
petal length (cm) 0.4300195278980297
petal width (cm) 0.4486112880533106


###**7.5 Boosting**

La idea general es hacer que el computador calcule múltiples estimadores, cada uno de estos intentando corregir al anterior, o en simple, superando al anterior. Los dos más conocidos son **AdaBoost** y **GradientBoosting**

####**a. AdaBoost**

In [12]:
from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1), n_estimators=200,
                            algorithm="SAMME.R", learning_rate=0.5)
ada_clf.fit(X_train, y_train)
#Si hace Overfit reduzco el número de estimadores

AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
          learning_rate=0.5, n_estimators=200, random_state=None)

In [13]:
y_pred = ada_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.896

####**b. GradienBoosting**

In [16]:
from sklearn.ensemble import GradientBoostingRegressor

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate=1.0)
gbrt.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=1.0, loss='ls', max_depth=2, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=3, n_iter_no_change=None, presort='auto',
             random_state=None, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False)

In [17]:
gbrt_slow = GradientBoostingRegressor(max_depth=2, n_estimators=200, learning_rate=0.1, random_state=42)
gbrt_slow.fit(X, y)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=2, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=200, n_iter_no_change=None, presort='auto',
             random_state=42, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False)

In [19]:
#número de estimadores óptimos
gbrt.n_estimators

3

**A continuación elementos no presentes en mi edición del libro (1era), los copié directo**

**[XGBoost](https://xgboost.readthedocs.io/en/latest/python/python_api.html)**

Iteraciones recursivas hasta alcanzar el menor error posible en la muestra de testeo

In [0]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


try:
    import xgboost
except ImportError as ex:
    print("Error: the xgboost library is not installed.")
    xgboost = None

In [25]:
if xgboost is not None:  # not shown in the book
    gbrt = xgboost.XGBRegressor(random_state=42)
    gbrt.fit(X_train, y_train)
    y_pred = gbrt.predict(X_test)
    val_error = mean_squared_error(y_test, y_pred) # Not shown
    print("Validation MSE:", val_error)           # Not shown

Validation MSE: 0.07582886953097483


In [28]:
if xgboost is not None:  # not shown in the book
    xgb_reg.fit(X_train, y_train,
                eval_set=[(X_test, y_test)], early_stopping_rounds=2)
    y_pred = xgb_reg.predict(X_test)
    val_error = mean_squared_error(y_test, y_pred)  # Not shown
    print("Validation MSE:", val_error)            # Not shown

[0]	validation_0-rmse:0.467243
Will train until validation_0-rmse hasn't improved in 2 rounds.
[1]	validation_0-rmse:0.438661
[2]	validation_0-rmse:0.414096
[3]	validation_0-rmse:0.392648
[4]	validation_0-rmse:0.375086
[5]	validation_0-rmse:0.359023
[6]	validation_0-rmse:0.346346
[7]	validation_0-rmse:0.334524
[8]	validation_0-rmse:0.324476
[9]	validation_0-rmse:0.316143
[10]	validation_0-rmse:0.309492
[11]	validation_0-rmse:0.30367
[12]	validation_0-rmse:0.299046
[13]	validation_0-rmse:0.294084
[14]	validation_0-rmse:0.290521
[15]	validation_0-rmse:0.287067
[16]	validation_0-rmse:0.283539
[17]	validation_0-rmse:0.27956
[18]	validation_0-rmse:0.276707
[19]	validation_0-rmse:0.272834
[20]	validation_0-rmse:0.27208
[21]	validation_0-rmse:0.269625
[22]	validation_0-rmse:0.268827
[23]	validation_0-rmse:0.267
[24]	validation_0-rmse:0.26684
[25]	validation_0-rmse:0.264461
[26]	validation_0-rmse:0.264167
[27]	validation_0-rmse:0.262823
[28]	validation_0-rmse:0.262675
[29]	validation_0-rmse:0.